In [13]:
import pandas as pd
import numpy as np
import torch
import playwright
import jmespath
from playwright.sync_api import sync_playwright
from twikit import Client
import json
import transformers
import openpyxl
#import sentencepiece

The command to use in order to have access to a tweet with its id is :

``twitter.com/anyuser/status/541278904204668929
``
With 541278904204668929 being an example of a tweet's id.

In [6]:
url = 'https://raw.githubusercontent.com/patriChiril/An-Annotated-Corpus-for-Sexism-Detection-in-French-Tweets/refs/heads/master/corpus_SexistContent.csv'
df_base = pd.read_csv(url, sep='\t')

In [7]:
df_base.columns = ['id','label']

In [8]:
df_base.to_csv('data_id.csv')

In [17]:
url = 'https://docs.google.com/spreadsheets/d/1eoC3NooDOwFoOvfweVLEZ-z-jCPZghUCVMUPTBkUDIk/export?format=csv&gid=1834088666'
df_whole = pd.read_csv(url)

In [23]:
df_whole.head()

,tweet_id,text
0,330425562201993216,NaN
1,326796299179548672,@MamzelleMNa Une très humble femme! #Ironie
2,334548844731826176,"BLOGUE - «Tsé, la parité homme-femme...» au se..."
3,332500824150392833,NaN
4,334424362033762304,Je suis une femme matérialiste et superficiell...


### Model's import

In [ ]:
from transformers import CamembertModel, CamembertTokenizer

In [ ]:
test_str = 'tarte au prout et au caca'

In [36]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("almanach/camembert-base")
model = AutoModelForMaskedLM.from_pretrained("almanach/camembert-base")


Some weights of the model checkpoint at almanach/camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
encoded_sentence = tokenizer.encode('tarte au caca')

In [ ]:
encoded_sentence = torch.tensor(encoded_sentence).unsqueeze(0)
embeddings, _ = camembert(encoded_sentence)

C:\Users\tpaqu\AppData\Local\Temp\ipykernel_11676\3093736408.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encoded_sentence = torch.tensor(encoded_sentence).unsqueeze(0)


ValueError: too many values to unpack (expected 2)

In [42]:
model(encoded_sentence)

ValueError: too many values to unpack (expected 2)